#### -----------------------------------------------------------------------------<br>Copyright (c) 2024, Lucid Vision Labs, Inc.
##### THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN  THE  SOFTWARE.<br>-----------------------------------------------------------------------------

In [ ]:
import time
from arena_api.system import system

#### Acquisition: TCP
>    Acquisition: RDMA.
	    This example introduces RDMA stream protocol. 
	    RDMA is a reliably connected transport protocol that transfers data 
	    from the camera to host memory without involving the CPU.
	    It features low - latency transfers and zero - copy.
	    A supported RDMA camera and capable NIC are required.

In [ ]:
"""
This function waits for the user to connect a device before raising an exception
"""

tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
            tries += 1
    else:
        print(f'Created {len(devices)} device(s)')
        device = devices[0]
        nodemap = device.nodemap
        tl_stream_nodemap = device.tl_stream_nodemap
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')


In [ ]:
print(f"Store initial values")
initial_acquisition_mode = nodemap.get_node("AcquisitionMode").value
width_initial = nodemap.get_node("Width").value
height_initial = nodemap.get_node("Height").value

##### Configure device settings

In [ ]:
nodemap.get_node("AcquisitionMode").value = "Continuous"

In [ ]:
tl_stream_nodemap["StreamBufferHandlingMode"].value = "NewestOnly"

#### Acquisition: TCP
>	The TransportStreamProtocol node can tell the camera to use the RDMA datastream engine. When
>	set to RDMA - Arena will switch to using the RDMA datastream engine. 
>	There is no further necessary configuration, though to achieve maximum throughput 
>   users may want to set the "DeviceLinkThroughputReserve" to 0 and 
>   also set the stream channel packet delay "GevSCPD" to 0.

In [ ]:
print('Set Transport Stream Protocol to RDMA\n')

try:
    nodemap["TransportStreamProtocol"].value = "RDMA"
except:
    print("Connected camera does not support RDMA stream")

In [ ]:
print(f'Getting \'Width\' and \'Height\' Nodes')
nodes = device.nodemap.get_node(['Width', 'Height'])

print('Setting \'Width\' and \'Height\' Nodes value to their '
      'max values')
nodes['Width'].value = nodes['Width'].max
nodes['Height'].value = nodes['Height'].max


In [ ]:
number_of_buffers = 10

device.start_stream(number_of_buffers)
print(f'Stream started with {number_of_buffers} buffers')

print(f'Get {number_of_buffers} buffers in a list')
buffers = device.get_buffer(number_of_buffers)
print("Success")

'''
Print image buffer info
    Buffers contain image data.
    Image data can also be copied and converted using BufferFactory.
    That is necessary to retain image data, as we must also requeue the buffer.
'''
for count, buffer in enumerate(buffers):
    print(f'\tbuffer{count:{2}} received | '
        f'Width = {buffer.width} pxl, '
        f'Height = {buffer.height} pxl, '
        f'Pixel Format = {buffer.pixel_format.name}')

device.requeue_buffer(buffers)
print(f'Requeued {number_of_buffers} buffers')

device.stop_stream()
print(f'Stream stopped')

In [ ]:
nodemap.get_node("AcquisitionMode").value = initial_acquisition_mode
nodemap.get_node("Width").value = width_initial
nodemap.get_node("Height").value = height_initial

system.destroy_device()
print('Destroyed all created devices')